<a href="https://colab.research.google.com/github/xydeep21/Deep-Learning-Data-Mining-model/blob/main/Simple_self_built_neural_network_and_the_application_of_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Forecasting the Demand of Purchasing during the Christmas Vacation 
# Using simple neural network

## Business Objectives : Demand Forecasting

This case study aims to identify the demand of purchasing during the Christmas vacation. The cookie store can use the prediction result to send coupons to those customers with lower desire to purchase and thus improve the sale during the Christmas Vacation. It can also be used to foresee how many cookies are needed in the coming Christmas Vacation and therefore help the store better prepare for the busy season.

In [24]:
#Mount with Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **1.Explore the dataset and select features** 

In [25]:
import numpy as np
import pandas as pd 

#import the data as a pandas dataframe
cookie_data = pd.read_excel("/content/drive/My Drive/All Practice Files Lab-1/Data/cookie_business.xlsx", sheet_name="Sheet1")

In [26]:
cookie_data.columns

Index(['Age', 'Postcode', 'Gender', 'Favourite Cookie',
       'Cookies bought frequency during Chrismas vaccations in the last 5 years'],
      dtype='object')

In [27]:
#View the head of the data
cookie_data.head()

,Age,Postcode,Gender,Favourite Cookie,Cookies bought frequency during Chrismas vaccations in the last 5 years
0,22,2010,0,1,0.08
1,30,2010,0,1,0.24
2,60,2000,1,1,0.04
3,53,2010,1,1,0.04
4,37,2000,0,1,0.08


In [28]:
cookie_data=cookie_data.drop("Postcode",axis=1)
cookie_data=cookie_data.drop("Favourite Cookie",axis=1)
cookie_data["Age"]/=100
cookie_data.head()

,Age,Gender,Cookies bought frequency during Chrismas vaccations in the last 5 years
0,0.22,0,0.08
1,0.30,0,0.24
2,0.60,1,0.04
3,0.53,1,0.04
4,0.37,0,0.08


In [29]:
#What's the demension of the data
cookie_data.shape

(46, 3)

In [30]:
#Column Description is shown below
pd.set_option('max_colwidth', 100)
pd.read_csv('/content/drive/My Drive/All Practice Files Lab-1/Data/Cookies Column Description.csv', sep = ';', index_col = 0)

,Description
Column Name,
Age,Customer Age
Postcode,Customer Postcode
Gender,"(0,1) 1 for Female"
Favorite Cookie,Favorite Cookie Type
Cookies Purchase Frequency,Cookies purchase frequency during Christmas vacations in the last 5 years


# **2.Plese design a self-built neural network model** </font>

In [31]:
#Split the data into training(80%) and testing(20%) sets
all_index = np.array(cookie_data.index)
train_index = np.random.choice(all_index,size=int(46*0.8),replace= False)
test_index = np.setdiff1d(all_index,train_index)
train_data = cookie_data.loc[train_index,]
test_data = cookie_data.loc[test_index,]
test_data.head()

,Age,Gender,Cookies bought frequency during Chrismas vaccations in the last 5 years
4,0.37,0,0.08
12,0.42,0,0.04
17,0.21,1,0.04
26,0.29,0,0.12
29,0.29,1,0.12


In [35]:
#Design a three layer neural network by using prelu method
#Thie neural networkd uses prelu method and have 2 hidden layers with 5 nodes in hidden layer1 and 1 nodes in hidden layer2
def my_net(train_data,test_data):
    def prelu(x,deriv=False):
        if(deriv==True):
            return (x>0)+(x<=0)*0.01
        return np.multiply((x>0),x) + np.multiply((x<=0)*0.01,x)

    X =  train_data.iloc[:,0:2].values  
    y = train_data.iloc[:,2].values
    y = np.reshape(y,(train_data.shape[0],1))
    w0 = 2*np.random.random((2,5))-1
    w1 = 2*np.random.random((5,1))-1
    for j in range(100):
        a0 = X
        a1 = prelu(np.dot(a0,w0))
        a2 = prelu(np.dot(a1,w1))
        a2_error = y - a2
        a2_delta = a2_error *prelu(a2,deriv=True)
        a1_error = a2_delta.dot(w1.T)
        a1_delta = a1_error * prelu(a1,deriv= True)
        w1 += a1.T.dot(a2_delta)
        w0 += a0.T.dot(a1_delta)
    
    print("Training MSE: ",((a2-y)**2).sum()/len(a2))
    X = train_data.iloc[:,0:2].values            
    y = train_data.iloc[:,2].values
    a0 = X
    a1 = prelu(np.dot(a0,w0))
    a2 = prelu(np.dot(a1,w1))
    a2_error = y - a2
    
    return(np.mean(a2_error**2))
    #return(np.mean(abs(a2_error)))

my_score = my_net(train_data,test_data)
print('Test MSE: ' + str(my_score))

Training MSE:  0.03592091799515932
Test MSE: 0.03313789909880962


# **3.Plese build a neural network model using Keras** </font>




In [34]:
from keras import models
from keras import layers
# The network architecture
# This nerual network has the same structure as the self-built neural network

def keras_net(train_data, test_data):
    network = models.Sequential()
    network.add(layers.Dense(5, activation='sigmoid', input_shape=(2,)))
    network.add(layers.Dense(1, activation='softmax'))
    # The compilation step
    network.compile(optimizer='SGD',loss = 'binary_crossentropy',metrics=['accuracy']) 
    # Fit the model
    train_info = train_data.iloc[:,0:2].values            
    train_labels = train_data.iloc[:,2].values
    history=network.fit(train_info, train_labels, epochs=100, verbose=0)
    print("Minimum Training mse: ",min(history.history["loss"]))
    # Get the accuracy rate
    test_info = test_data.iloc[:,0:2].values            
    test_labels = test_data.iloc[:,2].values
    #test_loss, test_mse = network.evaluate(test_info, test_labels)
    pred=network.predict(test_info)
    test_mse=np.mean((pred-test_labels)**2)
    return(test_mse)

keras_score = keras_net(train_data,test_data)
print('Keras Score: ' + str(keras_score))

Minimum Training mse:  12.504374504089355
Keras Score: 0.8126399999999999


### **4.Compare the two neural networks using K-fold validation**

In [36]:
#Split data into 3 folds
all_index = np.array(cookie_data.index)
all_index = np.random.choice(all_index,size=46,replace=False)
fold1_index = all_index[0:15]
fold2_index = all_index[15:30]
fold3_index = all_index[30:46]
fold1_data = cookie_data.loc[fold1_index,]
fold2_data = cookie_data.loc[fold2_index,]
fold3_data = cookie_data.loc[fold3_index,]

#Get the average validation score of the nerual network.
train_data1 = cookie_data.loc[np.union1d(fold1_index,fold2_index),]
test_data1 = fold3_data
score1 = my_net(train_data1,test_data1)

train_data2 = cookie_data.loc[np.union1d(fold1_index,fold3_index),]
test_data2 = fold2_data
score2 = my_net(train_data2,test_data2)

train_data3 = cookie_data.loc[np.union1d(fold2_index,fold3_index),]
test_data3 = fold1_data
score3 = my_net(train_data3,test_data3)

my_score = np.mean([score1,score2,score3])

#Get the average validation score of the nerual network built using Keras.
score4 = keras_net(train_data1,test_data1)
score5 = keras_net(train_data2,test_data2)
score6 = keras_net(train_data3,test_data3)

keras_score = np.mean([score4,score5,score6])

#Output
print('My Test MSE: ' + str(my_score) + '\n' + 'Keras Test MSE: ' + str(keras_score))

Training MSE:  0.041537139544545715
Training MSE:  0.03766225511143457
Training MSE:  0.015697392474142757
Minimum Training mse:  12.545038223266602
Minimum Training mse:  12.55356502532959
Minimum Training mse:  13.202887535095215
My Test MSE: 0.02900890566041538
Keras Test MSE: 0.7225333333333332
